In [1]:
# %%bash

# # cd /models && git fetch origin && git checkout origin/tf/transformer-api && pip install .
# cd /models && git checkout main && git pull origin main && pip install .
# cd /core && git checkout main && git pull origin main && pip install .
# cd /nvtabular && git checkout main && git pull origin main && pip install .
# cd /systems && git checkout main && git pull origin main && pip install .
# cd /dataloader && git checkout main && git pull origin main && pip install .

# ---
# pip install matplotlib

In [2]:
%%bash

rm -rf ecom_dataset
mkdir -p ecom_dataset

pip install gdown
# gdown https://drive.google.com/uc?id=1BvCHc4eXComuNK93bKhRM6cbg9y5p350  # <-- full dataset
gdown https://drive.google.com/uc?id=1NCFZ5ya3zyxPsrmupEoc9UEm4sslAddV
apt-get update -y
apt-get install unzip -y
unzip -d ecom_dataset "rees46_ecom_dataset_small_for_ci.zip"

Downloading...
From (uriginal): https://drive.google.com/uc?id=1NCFZ5ya3zyxPsrmupEoc9UEm4sslAddV
From (redirected): https://drive.google.com/uc?id=1NCFZ5ya3zyxPsrmupEoc9UEm4sslAddV&confirm=t&uuid=0dd96474-79af-47bb-9148-b96d64204e14
To: /workspace/T4Rec_repro/rees46_ecom_dataset_small_for_ci.zip
100%|██████████| 43.4M/43.4M [00:12<00:00, 3.62MB/s]


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.
Archive:  rees46_ecom_dataset_small_for_ci.zip
   creating: ecom_dataset/0001/
  inflating: ecom_dataset/0001/valid.parquet  
 extracting: ecom_dataset/0001/.zip  
  inflating: ecom_dataset/0001/train.parquet  
  inflating: ecom_dataset/0001/test.parquet  
   creating: ecom_dataset/0002/
  inflating: ecom_dataset/0002/valid.parquet  
  inflating: ecom_dataset/0002/train.parquet  
  inflating: ecom_dataset/0002/test.parquet  


In [3]:
# !cd /dataloader && git checkout main && git pull origin main && git checkout ce2215d8f871d0fb8c71900f7b914a226aea7c24 && pip install .

In [4]:
# !cd /core && git checkout main && git pull origin main && pip install .

In [5]:
# %%writefile /core/merlin/dag/graph.py

# #
# # Copyright (c) 2022, NVIDIA CORPORATION.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.
# #

# import logging
# from collections import deque
# from typing import Dict, Optional

# from merlin.dag.node import (
#     Node,
#     _combine_schemas,
#     iter_nodes,
#     postorder_iter_nodes,
#     preorder_iter_nodes,
# )
# from merlin.schema import Schema

# LOG = logging.getLogger("merlin")


# class Graph:
#     """
#     Represents an DAG composed of Nodes, each of which contains an operator that
#     transforms dataframes or dataframe-like data
#     """

#     def __init__(self, output_node: Node, subgraphs: Optional[Dict[str, Node]] = None):
#         self.output_node = output_node
#         self.subgraphs = subgraphs or {}

#         parents_with_deps = self.output_node.parents_with_dependencies
#         parents_with_deps.append(output_node)

#         for name, sg in self.subgraphs.items():
#             if sg not in parents_with_deps:
#                 raise ValueError(
#                     f"The output node of subgraph {name} does not exist in the provided graph."
#                 )

#     def subgraph(self, name: str) -> "Graph":
#         if name not in self.subgraphs.keys():
#             raise ValueError(f"No subgraph named {name}. Options are: {self.subgraphs.keys()}")
#         return Graph(self.subgraphs[name])

#     @property
#     def input_dtypes(self):
#         if self.input_schema:
#             return {
#                 name: col_schema.dtype
#                 for name, col_schema in self.input_schema.column_schemas.items()
#             }
#         else:
#             return {}

#     @property
#     def output_dtypes(self):
#         if self.output_schema:
#             return {
#                 name: col_schema.dtype
#                 for name, col_schema in self.output_schema.column_schemas.items()
#             }
#         else:
#             return {}

#     @property
#     def column_mapping(self):
#         nodes = preorder_iter_nodes(self.output_node)
#         column_mapping = self.output_node.column_mapping
#         for node in list(nodes)[1:]:
#             node_map = node.column_mapping
#             for output_col, input_cols in column_mapping.items():
#                 early_inputs = []
#                 for input_col in input_cols:
#                     early_inputs += node_map.get(input_col, [input_col])
#                 column_mapping[output_col] = early_inputs

#         return column_mapping

#     def construct_schema(self, root_schema: Schema, preserve_dtypes=False) -> "Graph":
#         """
#         Given the schema of a dataset to transform, determine the output schema of the graph

#         Parameters
#         ----------
#         root_schema : Schema
#             The schema of a dataset to be transformed with this DAG
#         preserve_dtypes : bool, optional
#             Whether to keep any dtypes that may already be present in the schemas, by default False

#         Returns
#         -------
#         Graph
#             This DAG after the schemas have been filled in
#         """
#         nodes = list(postorder_iter_nodes(self.output_node))

#         self._compute_node_schemas(root_schema, nodes, preserve_dtypes)
#         # self._validate_node_schemas(root_schema, nodes, preserve_dtypes)

#         return self

#     def _compute_node_schemas(self, root_schema, nodes, preserve_dtypes=False):
#         for node in nodes:
#             node.compute_schemas(root_schema, preserve_dtypes=preserve_dtypes)

#     def _validate_node_schemas(self, root_schema, nodes, strict_dtypes=False):
#         for node in nodes:
#             node.validate_schemas(root_schema, strict_dtypes=strict_dtypes)

#     @property
#     def input_schema(self):
#         # leaf_node input and output schemas are the same (aka selection)
#         return _combine_schemas(self.leaf_nodes)

#     @property
#     def leaf_nodes(self):
#         return [node for node in postorder_iter_nodes(self.output_node) if not node.parents]

#     @property
#     def output_schema(self):
#         return self.output_node.output_schema

#     def _input_columns(self):
#         input_cols = []
#         for node in iter_nodes([self.output_node]):
#             upstream_output_cols = []

#             for upstream_node in node.parents_with_dependencies:
#                 upstream_output_cols += upstream_node.output_columns.names

#             upstream_output_cols = _get_unique(upstream_output_cols)
#             input_cols += list(set(node.input_columns.names) - set(upstream_output_cols))

#         return _get_unique(input_cols)

#     def remove_inputs(self, to_remove):
#         """
#         Removes columns from a Graph

#         Starting at the leaf nodes, trickle down looking for columns to remove,
#         when found remove but then must propagate the removal of any other
#         output columns derived from that column.

#         Parameters
#         -----------
#         graph : Graph
#             The graph to remove columns from
#         to_remove : array_like
#             A list of input column names to remove from the graph

#         Returns
#         -------
#         Graph
#             The same graph with columns removed
#         """
#         nodes_to_process = deque([(node, to_remove) for node in self.leaf_nodes])

#         while nodes_to_process:
#             node, columns_to_remove = nodes_to_process.popleft()
#             if node.input_schema and len(node.input_schema):
#                 output_columns_to_remove = node.remove_inputs(columns_to_remove)

#                 for child in node.children:
#                     nodes_to_process.append(
#                         (child, list(set(to_remove + output_columns_to_remove)))
#                     )

#                     if not len(node.input_schema):
#                         node.remove_child(child)

#             # remove any dependencies that do not have an output schema
#             node.dependencies = [
#                 dep for dep in node.dependencies if dep.output_schema and len(dep.output_schema)
#             ]

#             if not node.input_schema or not len(node.input_schema):
#                 for parent in node.parents:
#                     parent.remove_child(node)
#                 for dependency in node.dependencies:
#                     dependency.remove_child(node)
#                 del node

#         return self

#     @classmethod
#     def get_nodes_by_op_type(cls, nodes, op_type):
#         return set(node for node in iter_nodes(nodes) if isinstance(node.op, op_type))


# def _get_schemaless_nodes(nodes):
#     schemaless_nodes = []
#     for node in iter_nodes(nodes):
#         if node.input_schema is None:
#             schemaless_nodes.append(node)

#     return set(schemaless_nodes)


# def _get_unique(cols):
#     # Need to preserve order in unique-column list
#     return list({x: x for x in cols}.keys())

In [6]:
# !cd /core && pip install .

In [7]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np

import tensorflow as tf

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm

2023-04-13 11:21:28.090236: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-04-13 11:21:30.471061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 11:21:30.471514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 11:21:30.471678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-04-13 11:21:30.757567: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 11:21:30.758435: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 11:21:30.758639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-13 11:21:30.758792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

In [8]:
train = Dataset("ecom_dataset/0001/train.parquet")
valid = Dataset("ecom_dataset/0002/test.parquet")

In [9]:
target = 'sess_pid_seq'
seq_name = target

In [10]:
# a couple of starter hyperparams

d_model = 192
n_layer = 3
n_head = 16
batch_size = 128
learning_rate = 0.0006667377132554976
n_epoch = 1
item_embedding_dim = 448 
item_id_embeddings_init_std = 3

In [11]:
# seq_name = 'seq'
# target = seq_name

In [12]:
from nvtabular.inference.triton import export_tensorflow_ensemble
from nvtabular import Workflow
from nvtabular.ops import Categorify, Rename

In [13]:
ops = ['sess_pid_seq'] >> Categorify(dtype=np.int32) #>> Rename(name=seq_name)

wf = Workflow(ops)

train = wf.fit_transform(train)
valid = wf.transform(valid)

In [14]:
# cat rees46_schema_modified.pbtxt

In [15]:
# %%writefile rees46_schema_modified_2.pbtxt

# feature {
#   name: "seq"
#   value_count {
#     min: 2
#   }
#   type: INT
#   int_domain {
#     name: "seq"
#     min: 1
#     max: 390000
#     is_categorical: true
#   }
#   annotation {
#     tag: "item_id"
#     tag: "list"
#     tag: "categorical"
#     tag: "item"
#   }
# }

In [16]:
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

def get_model():
    mlp_block = mm.MLPBlock(
                    [d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    schema = TensorflowMetadata.from_proto_text_file(
        './',
        file_name='rees46_schema_modified.pbtxt'
    ).to_merlin_schema()

    train.schema = schema
    
    schema_model = schema.select_by_tag(Tags.ITEM_ID)
    input_block = mm.InputBlockV2(
        schema_model,
        categorical=mm.Embeddings(
                schema_model.select_by_tag(Tags.CATEGORICAL),
                dim=item_embedding_dim,
                sequence_combiner=None,
            )
        )

    train.schema = train.schema.select_by_name(seq_name)

    xlnet_block = mm.XLNetBlock(d_model=d_model, n_head=n_head, n_layer=n_layer)

    dense_block = mm.SequentialBlock(
        input_block,
        mlp_block,
        xlnet_block
    )

    mlp_block2 = mm.MLPBlock(
                    [item_embedding_dim],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    prediction_task = mm.CategoricalOutput(
        to_call=input_block["categorical"][target],
    )

    model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
    )

    model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
                  metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[20])
                 )
    return model_transformer, xlnet_block

In [17]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

# model_transformer.save('t4rec_model')

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2023-04-13 11:21:38.342588: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:436: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 174720448 elements. This may consume a large amount of memory.
  warnings.warn(


84/84 [==============================] - 4s 27ms/step - loss: 8.5848 - recall_at_20: 0.2229 - mrr_at_20: 0.0736 - ndcg_at_20: 0.1066 - map_at_20: 0.0736 - precision_at_20: 0.0111 - regularization_loss: 0.0000e+00 - loss_batch: 8.5971


{'loss': 8.584781646728516,
 'recall_at_20': 0.2308632731437683,
 'mrr_at_20': 0.07471762597560883,
 'ndcg_at_20': 0.10908268392086029,
 'map_at_20': 0.07471762597560883,
 'precision_at_20': 0.011543160304427147,
 'regularization_loss': 0.0,
 'loss_batch': 9.130510330200195}

In [18]:
from merlin.systems.dag.ops.workflow import TransformWorkflow
from merlin.systems.dag.ops.tensorflow import PredictTensorflow

serving_operators = [seq_name] >> TransformWorkflow(wf) >> PredictTensorflow(model_transformer)

  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.
  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.
  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.
  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


INFO:tensorflow:Assets written to: /tmp/tmpkph1f3_r/model.savedmodel/assets


INFO:tensorflow:Assets written to: /tmp/tmpkph1f3_r/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:100: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_obj

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [19]:
# import merlin.models.tf as mm
# import tensorflow as tf
# tf_model_path = os.path.join('t4rec_model')

# model = tf.keras.models.load_model(tf_model_path)

In [20]:
%%bash

rm -rf /workspace/models_for_benchmarking
mkdir -p /workspace/models_for_benchmarking

In [21]:
train.schema.select_by_name('sess_pid_seq')

,name,tags,dtype,is_list,is_ragged,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,sess_pid_seq,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ID, Tags.IT...","DType(name='int64', element_type=<ElementType....",True,True,1,390000,sess_pid_seq,2,None


In [22]:
from merlin.systems.dag.ensemble import Ensemble

ensemble = Ensemble(serving_operators, wf.input_schema)
ens_conf, node_confs = ensemble.export("/workspace/models_for_benchmarking")

  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


INFO:tensorflow:Assets written to: /workspace/models_for_benchmarking/1_predicttensorflowtriton/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: /workspace/models_for_benchmarking/1_predicttensorflowtriton/1/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:100: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argume

In [23]:
ls /workspace/models_for_benchmarking/1

ls: cannot access '/workspace/models_for_benchmarking/1': No such file or directory


In [24]:
import nvtabular.inference.triton as nvt_triton
import tritonclient.grpc as grpcclient
import subprocess

subprocess.Popen(['tritonserver', '--model-repository=/workspace/models_for_benchmarking/'])

I0413 11:24:28.716029 1527 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7f7f2a000000' with size 268435456
I0413 11:24:28.716361 1527 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0413 11:24:28.718446 1527 model_lifecycle.cc:459] loading: 0_transformworkflowtriton:1
I0413 11:24:28.718465 1527 model_lifecycle.cc:459] loading: 1_predicttensorflowtriton:1
I0413 11:24:28.718478 1527 model_lifecycle.cc:459] loading: executor_model:1
I0413 11:24:28.924940 1527 tensorflow.cc:2536] TRITONBACKEND_Initialize: tensorflow
I0413 11:24:28.924955 1527 tensorflow.cc:2546] Triton TRITONBACKEND API version: 1.10
I0413 11:24:28.924960 1527 tensorflow.cc:2552] 'tensorflow' TRITONBACKEND API version: 1.10
I0413 11:24:28.924962 1527 tensorflow.cc:2576] backend configuration:
{"cmdline":{"auto-complete-config":"true","min-compute-capability":"6.000000","backend-directory":"/opt/tritonserver/backends","default-max-batch-size":"4"}}
2023-04-13 11:24

2023-04-13 11:24:43.330311: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /workspace/models_for_benchmarking/1_predicttensorflowtriton/1/model.savedmodel
2023-04-13 11:24:43.395816: I tensorflow/cc/saved_model/loader.cc:325] SavedModel load for tags { serve }; Status: success: OK. Took 710922 microseconds.
I0413 11:24:43.395921 1527 python_be.cc:1856] TRITONBACKEND_ModelInstanceInitialize: 0_transformworkflowtriton_0 (GPU device 0)
I0413 11:24:43.396107 1527 model_lifecycle.cc:694] successfully loaded '1_predicttensorflowtriton' version 1
2023-04-13 11:24:44.668497: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 11:24:46.525315: I tensorflow/strea

2023-04-13 11:25:37.504455: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


In [ ]:
# !pkill triton

In [ ]:
import tritonhttpclient
try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))
triton_client.is_server_live()

In [ ]:
validation_data.iloc[]

In [ ]:
from merlin.systems.triton import convert_df_to_triton_input

validation_data = valid.compute()
inputs = convert_df_to_triton_input(wf.input_schema, validation_data.iloc[:1])

In [ ]:
inputs[0].name()

In [ ]:
inputs[0].shape()

In [ ]:
inputs[1].name()

In [ ]:
inputs[1].shape()

In [ ]:
validation_data.iloc[:1]

In [ ]:
wf.input_schema

In [ ]:
import tritonclient.grpc as grpcclient

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer('1_predicttensorflowtriton', inputs)

In [ ]:
response.get_output('sess_pid_seq/categorical_output')

In [16]:
from merlin.systems.dag.ensemble import Ensemble

ensemble = Ensemble(serving_operators, train.schema)
ens_conf, node_confs = ensemble.export("/workspace/models_for_benchmarking")

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


...

  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


  (_feature_shapes): Dict(
    (sess_pid_seq): TensorShape([128, None, 1])
  )
  (_feature_dtypes): Dict(
    (sess_pid_seq): tf.int32
  )
), because it is not built.


INFO:tensorflow:Assets written to: /workspace/models_for_benchmarking/0_predicttensorflowtriton/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: /workspace/models_for_benchmarking/0_predicttensorflowtriton/1/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:83: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argumen

In [16]:
cat /workspace/models_for_benchmarking/0_predicttensorflowtriton/config.pbtxt

name: "0_predicttensorflowtriton"
platform: "tensorflow_savedmodel"
input {
  name: "sess_pid_seq"
  data_type: TYPE_INT32
  dims: -1
  dims: 1
}
input {
  name: "sess_pid_seq_1"
  data_type: TYPE_INT32
  dims: -1
  dims: 1
}
output {
  name: "sess_pid_seq/categorical_output"
  data_type: TYPE_FP32
  dims: -1
  dims: 390001
}
parameters {
  key: "TF_GRAPH_TAG"
  value {
    string_value: "serve"
  }
}
parameters {
  key: "TF_SIGNATURE_DEF"
  value {
    string_value: "serving_default"
  }
}
backend: "tensorflow"


In [18]:
%%writefile /workspace/models_for_benchmarking/0_predicttensorflowtriton/config.pbtxt

name: "0_predicttensorflowtriton"
platform: "tensorflow_savedmodel"
input {
  name: "sess_pid_seq"
  data_type: TYPE_INT32
  dims: -1
  dims: 1
}
input {
  name: "sess_pid_seq_1"
  data_type: TYPE_INT32
  dims: -1
  dims: 1
}
output {
  name: "sess_pid_seq/categorical_output"
  data_type: TYPE_FP32
  dims: -1
  dims: 390001
}
parameters {
  key: "TF_GRAPH_TAG"
  value {
    string_value: "serve"
  }
}
parameters {
  key: "TF_SIGNATURE_DEF"
  value {
    string_value: "serving_default"
  }
}
backend: "tensorflow"

Overwriting /workspace/models_for_benchmarking/0_predicttensorflowtriton/config.pbtxt


In [17]:
cat /workspace/models_for_benchmarking/executor_model/config.pbtxt

name: "executor_model"
platform: "merlin_executor"
input {
  name: "sess_pid_seq__values"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
input {
  name: "sess_pid_seq__lengths"
  data_type: TYPE_INT32
  dims: -1
  dims: -1
}
output {
  name: "sess_pid_seq/categorical_output"
  data_type: TYPE_FP32
  dims: -1
  dims: 390001
}
backend: "python"


In [19]:
%%writefile /workspace/models_for_benchmarking/executor_model/config.pbtxt

name: "executor_model"
platform: "merlin_executor"
input {
  name: "sess_pid_seq__values"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
input {
  name: "sess_pid_seq__nnzs"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
output {
  name: "sess_pid_seq/categorical_output"
  data_type: TYPE_FP32
  dims: -1
  dims: 390001
}
backend: "python"

Overwriting /workspace/models_for_benchmarking/executor_model/config.pbtxt


In [21]:
cat /workspace/models_for_benchmarking/executor_model/1/model.py

# Copyright (c) 2022, NVIDIA CORPORATION. All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions
# are met:
#  * Redistributions of source code must retain the above copyright
#    notice, this list of conditions and the following disclaimer.
#  * Redistributions in binary form must reproduce the above copyright
#    notice, this list of conditions and the following disclaimer in the
#    documentation and/or other materials provided with the distribution.
#  * Neither the name of NVIDIA CORPORATION nor the names of its
#    contributors may be used to endorse or promote products derived
#    from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS ``AS IS'' AND ANY
# EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
# PURPOSE ARE DISCLAIM

In [26]:
cat /workspace/models_for_benchmarking/executor_model/1/ensemble/metadata.json

{"versions": {"python": "3.8.10 (default, Nov 14 2022, 12:59:47) \n[GCC 9.4.0]"}, "generated_timestamp": 1679017581}

In [ ]:
# %%writefile /workspace/models_for_benchmarking/t4r_pytorch_pt/config.pbtxt

# name: "t4r_pytorch_pt"
# input {
#   name: "sess_pid_seq__values"
#   data_type: TYPE_INT64
#   dims: -1
#   dims: 1
# }
# input {
#   name: "sess_pid_seq__nnzs"
#   data_type: TYPE_INT64
#   dims: -1
#   dims: 1
# }
# output {
#   name: "output"
#   data_type: TYPE_FP32
#   dims: -1
#   dims: 20
# }
# backend: "python"